In [18]:
# Jupyter Notebook for Testing Erigon Service with web3

# This notebook assumes that the Docker services are already running.
# It uses the web3 library to interact with the Erigon service and perform some basic tests.

import json
import requests
from web3 import Web3
from web3.middleware import geth_poa_middleware
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

# Connect to the Erigon service
erigon_url = "http://erigon:8545"
web3 = Web3(Web3.HTTPProvider(erigon_url))

# Inject the poa compatibility middleware to the innermost layer
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

# Check if the connection is successful
if web3.is_connected():
    print("Successfully connected to Erigon")
else:
    print("Failed to connect to Erigon")

# Define a helper function to print formatted JSON
def print_json(data):
    print(json.dumps(data, indent=2))

# Function to convert AttributeDict and HexBytes to dict
def attrdict_to_dict(attrdict):
    if isinstance(attrdict, AttributeDict):
        return {k: attrdict_to_dict(v) for k, v in attrdict.items()}
    elif isinstance(attrdict, dict):
        return {k: attrdict_to_dict(v) for k, v in attrdict.items()}
    elif isinstance(attrdict, list):
        return [attrdict_to_dict(i) for i in attrdict]
    elif isinstance(attrdict, HexBytes):
        return attrdict.hex()
    else:
        return attrdict

try:
    # Get the latest block number
    latest_block_number = web3.eth.block_number
    print(f"Latest Block Number: {latest_block_number}")

    # Get the latest block details
    latest_block = web3.eth.get_block(latest_block_number)
    latest_block_dict = attrdict_to_dict(latest_block)
    print("Latest Block:")
    print_json(latest_block_dict)

    # Get information about a specific block by number
    block_number = latest_block_number - 10  # Example: 10 blocks before the latest
    block = web3.eth.get_block(block_number)
    block_dict = attrdict_to_dict(block)
    print(f"Block {block_number}:")
    print_json(block_dict)

    # Get the latest Ethereum gas price
    gas_price = web3.eth.gas_price
    print(f"Latest Gas Price: {gas_price}")

    # Get information about a specific transaction by hash
    # This assumes that there is at least one transaction in the latest block
    if len(latest_block['transactions']) > 0:
        tx_hash = latest_block['transactions'][0]
        transaction = web3.eth.get_transaction(tx_hash)
        transaction_dict = attrdict_to_dict(transaction)
        print(f"Transaction {tx_hash}:")
        print_json(transaction_dict)
    else:
        print("No transactions found in the latest block")

    # Call the net_version method to get the network ID
    network_id = web3.net.version
    print(f"Network ID: {network_id}")

except Exception as e:
    print(f"An error occurred: {e}")

# Test Erigon service by sending a JSON-RPC request directly
payload = {
    "jsonrpc": "2.0",
    "method": "eth_blockNumber",
    "params": [],
    "id": 1
}
try:
    response = requests.post(erigon_url, json=payload)
    response.raise_for_status()  # Raise an error for bad status codes
    result = response.json()
    print(f"eth_blockNumber result: {result}")
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")


Successfully connected to Erigon
Latest Block Number: 19870999
Latest Block:
{
  "baseFeePerGas": 5331066125,
  "blobGasUsed": 0,
  "difficulty": 0,
  "excessBlobGas": 393216,
  "proofOfAuthorityData": "0x6265617665726275696c642e6f7267",
  "gasLimit": 30000000,
  "gasUsed": 10575095,
  "hash": "0xa99015f464dad00cf251627681bfce6715664b6d35e4a668f2fe52f184095c1a",
  "logsBloom": "0x04a90513db23682b934a0158a2303a273985d944182709fea98d486386e0008395c41baa43000538ea053a218842216d8315d8f48828f90c412db18a093d10b03dca9d2c4b4b9b3d6ba2e8bfc04040aa850451e7416c7e6359268d4bd06350acd411ca00ca63223ec0cdf8d419312d55d730122248884e24307649dc45a908410b2093703dac91d140c00db00b22c4fe28298c018bc5808cc46b417aab304f3befcb1c3e089161560ab46ac49a92a50224c3a9020418689b8062a02003b93f42196a395e44132cde5842a09ec759aa648aba935f02302c10b20268e388dc60a904f075700182051027867ec154004a13547a9881c8cc28d9cc2d11881e5b1434",
  "miner": "0x95222290DD7278Aa3Ddd389Cc1E1d165CC4BAfe5",
  "mixHash": "0xcc4093ebdab1df36f1843472cb572